# Testing different SA methods 3/5
## nltk's vader

In [1]:
import csv

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer(remove_stopwords=False)

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Don't use irrelevant tweets here, thats too hard for Vader
    tweets = [(text, sentiment) for (topic, sentiment, id, text) in iterator \
              if sentiment == "positive" or sentiment == "negative" or sentiment == "neutral"]

In [2]:
"""
Copy of the NLTK-Vader demo, modified to use our own dataset, for comparable results
"""
from collections import defaultdict
from nltk.corpus import twitter_samples
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.metrics import (accuracy as eval_accuracy, precision as eval_precision,
    recall as eval_recall, f_measure as eval_f_measure)
from nltk.sentiment.util import json2csv_preprocess, parse_tweets_set

testing_tweets = tweets

vader_analyzer = SentimentIntensityAnalyzer()

gold_results = defaultdict(set)
test_results = defaultdict(set)
acc_gold_results = []
acc_test_results = []
labels = set()
num = 0
for i, (text, label) in enumerate(testing_tweets):
    labels.add(label)
    gold_results[label].add(i)
    acc_gold_results.append(label)
    score = vader_analyzer.polarity_scores(text)['compound']
    
    threshold = 0.8 # threshold with the highest accuracy
    if score <= -threshold:
        observed = 'negative'
    elif score > threshold:
        observed = 'positive'
    else:
        observed = "neutral"
        
    num += 1
    acc_test_results.append(observed)
    test_results[observed].add(i)
metrics_results = {}
for label in labels:
    accuracy_score = eval_accuracy(acc_gold_results,
        acc_test_results)
    metrics_results['Accuracy'] = accuracy_score
    precision_score = eval_precision(gold_results[label],
        test_results[label])
    metrics_results['Precision [{0}]'.format(label)] = precision_score
    recall_score = eval_recall(gold_results[label],
        test_results[label])
    metrics_results['Recall [{0}]'.format(label)] = recall_score
    f_measure_score = eval_f_measure(gold_results[label],
        test_results[label])
    metrics_results['F-measure [{0}]'.format(label)] = f_measure_score

for result in sorted(metrics_results):
        print('{0}: {1}'.format(result, metrics_results[result]))

Accuracy: 0.6813337916809901
F-measure [negative]: 0.09867172675521821
F-measure [neutral]: 0.8067653276955603
F-measure [positive]: 0.1711229946524064
Precision [negative]: 0.7428571428571429
Precision [neutral]: 0.6895554752439466
Precision [positive]: 0.4485981308411215
Recall [negative]: 0.052845528455284556
Recall [neutral]: 0.9719816607233825
Recall [positive]: 0.10572687224669604


In [6]:
# Funny how vader performs way better on the nltk dataset
# TODO look at code and dataset
import nltk.sentiment.util
nltk.sentiment.util.demo_vader_tweets()

Loaded 5000 tweets


Loaded 5000 tweets
Accuracy: 0.861
F-measure [neg]: 0.8524416135881105
F-measure [pos]: 0.8686200378071833
Precision [neg]: 0.9083710407239819
Precision [pos]: 0.8234767025089605
Recall [neg]: 0.803
Recall [pos]: 0.919
